To build/rebuild run: 
```
jbuilder build src-core/core.cma
jbuilder build src-core-pp/core_pp.cma
```

In [1]:
#directory "_build/default/src-core/";;
#directory "_build/default/src-core-pp/";;
#load "_build/default/src-core/core.cma";;
#load "_build/default/src-core-pp/core_pp.cma";;

In [2]:
let message_of_string str = str
    |> Stream.of_string 
    |> Parser_utils.split_into_key_value '|'
    |> Parser_utils.split_into_messages
    |> Stream.peek
    |> function Some m -> m | _ -> failwith "WAT"

val message_of_string : string -> (string * string) list = <fun>


In [3]:
let message = message_of_string "42=42|43=43|45=45|1=3|2=9|3=9|2=8|3=8|2=7|3=7|12=12|13=13|"

val message : (string * string) list =
  [("42", "42"); ("43", "43"); ("45", "45"); ("1", "3"); ("2", "9");
   ("3", "9"); ("2", "8"); ("3", "8"); ("2", "7"); ("3", "7"); ("12", "12");
   ("13", "13")]


# Checking my custom list utils for the parser

`Parser_utils.take` extracts a value given a tag, leavig rest of the list unmodified.

In [4]:
Parser_utils.take "1" message;;

- : string option * (string * string) list =
(Some "3",
 [("42", "42"); ("43", "43"); ("45", "45"); ("2", "9"); ("3", "9");
  ("2", "8"); ("3", "8"); ("2", "7"); ("3", "7"); ("12", "12"); ("13", "13")])


`Parser_utils.split_on_tag` splits message at a given tag

In [5]:
let leading, groups = Parser_utils.split_on_tag "1" message;;

val leading : (string * string) list =
  [("42", "42"); ("43", "43"); ("45", "45")]
val groups : (string * string) list =
  [("1", "3"); ("2", "9"); ("3", "9"); ("2", "8"); ("3", "8"); ("2", "7");
   ("3", "7"); ("12", "12"); ("13", "13")]


In [6]:
List.tl groups |> Parser_utils.cut_on_separator

- : (string * string) list list =
[[("2", "9"); ("3", "9")]; [("2", "8"); ("3", "8")];
 [("2", "7"); ("3", "7"); ("12", "12"); ("13", "13")]]


# Simple message with a single repeating group

In [11]:
open Parse_base_types 
open Parser_utils.Parser

In [50]:
(* Block and message types*)
type test_block1 = {
    tb_2 : int;
    tb_3 : int option
};; 

type test_message = {
    tm_42 : int;
    tm_repeating : test_block1 list;    
    tm_12 : int option;
    tm_13 : int 
}

type test_block1 = { tb_2 : int; tb_3 : int option; }
type test_message = {
  tm_42 : int;
  tm_repeating : test_block1 list;
  tm_12 : int option;
  tm_13 : int;
}


In [51]:
(* Block and message parsers *)
let parse_block1 msg =
    req msg "2" parse_int @@ fun msg tb_2 ->
    opt msg "3" parse_int @@ fun msg tb_3 ->
    ParseSuccess { tb_2; tb_3 } , msg

let parse_message msg =
    repeating msg "1" parse_block1 @@ fun msg tm_repeating ->
    req msg "42" parse_int @@ fun msg tm_42 -> 
    opt msg "12" parse_int @@ fun msg tm_12 -> 
    req msg "13" parse_int @@ fun msg tm_13 -> 
    ParseSuccess { tm_42; tm_12; tm_13; tm_repeating }, msg  
    

val parse_block1 :
  (string * string) list ->
  test_block1 Parser_utils.Parser.t * (string * string) list = <fun>
val parse_message :
  Parser_utils.Parser.msg ->
  test_message Parser_utils.Parser.t * Parser_utils.Parser.msg = <fun>


### Testing the parser and error reporing

In [52]:
"42=42|" ^
"1=3|" ^ "2=9|3=9|"
       ^ "2=8|3=8|"
       ^ "2=7|3=7|" ^
"12=12|13=13|" 
|> message_of_string |> parse_message 

- : test_message Parser_utils.Parser.t * Parser_utils.Parser.msg =
(ParseSuccess
  {tm_42 = 42;
   tm_repeating =
    [{tb_2 = 9; tb_3 = Some 9}; {tb_2 = 8; tb_3 = Some 8};
     {tb_2 = 7; tb_3 = Some 7}];
   tm_12 = Some 12; tm_13 = 13},
 [])


In [54]:
"42=42|" ^
"1=3|" ^ "2=9|3=9|"
       ^ "2=8|"
       ^ "2=7|3=7|" ^
"12=12|13=13|" 
|> message_of_string |> parse_message 

- : test_message Parser_utils.Parser.t * Parser_utils.Parser.msg =
(ParseSuccess
  {tm_42 = 42;
   tm_repeating =
    [{tb_2 = 9; tb_3 = Some 9}; {tb_2 = 8; tb_3 = None};
     {tb_2 = 7; tb_3 = Some 7}];
   tm_12 = Some 12; tm_13 = 13},
 [])


In [67]:
"42=42|" ^
"1=1|" ^ "2=9|" ^
"12=12|13=13|" 
|> message_of_string |> parse_message 

- : test_message Parser_utils.Parser.t * Parser_utils.Parser.msg =
(ParseSuccess {tm_42 = 42; tm_repeating = []; tm_12 = Some 12; tm_13 = 13},
 [("1", "1"); ("2", "9")])


In [56]:
"42=42|" ^
"1=3|" ^ "2=9|3=9|5=7|"
       ^ "2=8|"
       ^ "2=7|3=7|" ^
"12=12|13=13|" 
|> message_of_string |> parse_message 

- : test_message Parser_utils.Parser.t * Parser_utils.Parser.msg =
(RepeatingGroupOutOfOrder "1", [])


In [55]:
"42=42|" ^
"1=3|" ^ "2=9|3=9|"
       ^ "2=7|3=7|" ^
"12=12|13=13|" 
|> message_of_string |> parse_message 

- : test_message Parser_utils.Parser.t * Parser_utils.Parser.msg =
(IncorrectNumInGroupCount "1", [])


# Repeating group inside a repeting group

In [57]:
(* Block and message types*)
type test_block1 = {
    tb_2 : int;
    tb_3 : int option
};; 

type test_block4 = {
    tb_5 : int;
    tb_6 : test_block1 list;
    tb_7 : int option    
};; 

type test_message = {
    tm_42 : int;
    tm_repeating : test_block4 list;    
    tm_12 : int option;
    tm_13 : int 
}

type test_block1 = { tb_2 : int; tb_3 : int option; }
type test_block4 = {
  tb_5 : int;
  tb_6 : test_block1 list;
  tb_7 : int option;
}
type test_message = {
  tm_42 : int;
  tm_repeating : test_block4 list;
  tm_12 : int option;
  tm_13 : int;
}


In [58]:
(* Block and message parsers *)
let parse_block1 msg =
    req msg "2" parse_int @@ fun msg tb_2 ->
    opt msg "3" parse_int @@ fun msg tb_3 ->
    ParseSuccess { tb_2; tb_3 } , msg

let parse_block4 msg =
    repeating msg "4" parse_block1 @@ fun msg tb_6 ->
    req msg "5" parse_int @@ fun msg tb_5 ->
    opt msg "7" parse_int @@ fun msg tb_7 ->
    ParseSuccess {tb_5; tb_6; tb_7} , []

val parse_block1 :
  (string * string) list ->
  test_block1 Parser_utils.Parser.t * (string * string) list = <fun>
val parse_block4 :
  Parser_utils.Parser.msg ->
  test_block4 Parser_utils.Parser.t * Parser_utils.Parser.msg = <fun>


In [64]:
(* Block and message parsers *)
let parse_block1 msg =
    req msg "2" parse_int @@ fun msg tb_2 ->
    opt msg "3" parse_int @@ fun msg tb_3 ->
    ParseSuccess { tb_2; tb_3 } , msg

let parse_block4 msg =
    repeating msg "1" parse_block1 @@ fun msg tb_6 -> 
    req msg "5" parse_int @@ fun msg tb_5 ->
    opt msg "7" parse_int @@ fun msg tb_7 ->
    ParseSuccess {tb_5; tb_6=[]; tb_7} , []
    
let parse_message msg =
    repeating msg "4" parse_block4 @@ fun msg tm_repeating ->
    req msg "42" parse_int @@ fun msg tm_42 -> 
    opt msg "12" parse_int @@ fun msg tm_12 -> 
    req msg "13" parse_int @@ fun msg tm_13 -> 
    ParseSuccess { tm_42; tm_12; tm_13; tm_repeating }, msg  

val parse_block1 :
  (string * string) list ->
  test_block1 Parser_utils.Parser.t * (string * string) list = <fun>
val parse_block4 :
  Parser_utils.Parser.msg ->
  test_block4 Parser_utils.Parser.t * Parser_utils.Parser.msg = <fun>
val parse_message :
  Parser_utils.Parser.msg ->
  test_message Parser_utils.Parser.t * Parser_utils.Parser.msg = <fun>


In [65]:
"42=42|4=2|" ^
    "5=1|" ^
    "5=2|" ^
"13=1|"
|> message_of_string |> parse_message 

- : test_message Parser_utils.Parser.t * Parser_utils.Parser.msg =
(RequiredTagMissing "13", [])


In [48]:
( "42=42|4=2|" ^
    "5=1|" ^
    "5=2|" ^
"13=1"
)

- : string = "42=42|4=2|5=1|5=2|13=1"
